# Vanishing/Exploding gradient problem in rnn

Lets analyse the gradients of a simple rnn to understand the problem we encounter during training.

<img src='assets/vanish_grad1.png'>

RNNs are trained using 'Backpropagation through time (BPTT)', where the objective is to learn the parameters $W_{xh}, W_{hh}, W_{hy}$ by obtaining the error gradients with respect to those parameters through gradient desent.

Lets compute the gradient $\large\frac{\partial E} {\partial W}$ for the above RNN which has three stages. The weights are obtained by summing up the gradients obtained at each time step: $\sum\limits_{t}\large\frac{\partial E_{t}} {\partial W_{hh}}$. But lets only calculate $\large\frac{\partial E_{t+1}} {\partial W}$ for this exercise.

Lets do the 'computational graph gradient flow approach' to track the gradients as it flows back in various computational nodes. Go through [Gradient flow in neural network lesson]('http://localhost:8888/notebooks/intro/Gradient%20Flow.ipynb') for the details.

The equations involved:   
$$\begin{align}
z_{t} &= U x_{t} + W s_{t-1} \\    
s_{t} &= tanh(z_{t}) \\    
q_{t} &= V s_{t} \\    
o_{t} &= softmax(q_{t}) \\   
\end{align} $$


#### Backpropagating from the last output node:
* Since we need to calculate $E_{t+1}$ with respect to $W$, we will start with output node: $o_{t+1}$. Since the weights W is common for all time steps, we need to individualy calculate their gradient for each time step and add the results for the final gradient. Note all the notations below are in matrix form.
* ##### W gradient at time (t+1):
    + The error gradient $\large\frac{\partial E_{t+1}} {\partial o_{t+1}}$ is propagated back.
    + Next is the activation gate, with output $o_{t+1}$ and input $q_{t+1}$ , so the local gradient $\large\frac{\partial o_{t+1}} {\partial q_{t+1}}$ gets multiplied with the incoming gradient. Thus we get
    
    $$\frac{\partial E_{t+1}} {\partial o_{t+1}} . \frac{\partial o_{t+1}} {\partial q_{t+1}} $$
        
    + Now we encounter the multiplication gate with weights V and $s_{t+1}$, so the gradient towards $s_{t+1}$ is the product of the incoming gradient multiplied by V. The gradient at this point is 
    
    $$\frac{\partial E_{t+1}} {\partial o_{t+1}} . \frac{\partial o_{t+1}} {\partial q_{t+1}} . \small V $$
    
    + After that comes the activation unit whose output is $s_{t+1}$ and input is $z_{t+1}$. So the local gradient gets multipled with the incoming gradient. And we get,
    
    $$\frac{\partial E_{t+1}} {\partial o_{t+1}} . \frac{\partial o_{t+1}} {\partial q_{t+1}} . \small V . \frac{\partial s_{t+1}} {\partial z_{t+1}} $$
    
    + Next comes the multiplication gate where the gradient at W is obtained by multiplying the incoming gradient with the 'other input' $s_{t}$, while the gradient that flows through $s_{t}$ gets multiplied with W. So the gradient at W is:   
    
    $$\frac{\partial E_{t+1}} {\partial W} = 
    \frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    S_{t}$$ 
    
    and the other gradient that flows through $s_{t+1}$ is   
    $$\frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    \small W$$

* ##### W gradient at time (t):
    + Here the next node is the activation unit, so the local gradient gets multiplied with the incoming gradient from the previous step. We get, 
    
    $$\frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    \small W . 
    \frac{\partial s_{t}} {\partial z_{t}}$$
    
    + Next comes the multiplication gate where the gradient at W is obtained by multiplying the incoming gradient with the 'other input' $s_{t-1}$, while the gradient that flows through $s_{t-1}$ gets multiplied with W. So the gradient at W is:
    
     $$\frac{\partial E_{t+1}} {\partial W} =
     \frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    \small W . 
    \frac{\partial s_{t}} {\partial z_{t}}. S_{t-1}$$ 
     and the other gradient that flows through $s_{t-1}$ is 
     
     $$\frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    \small W . 
    \frac{\partial s_{t}} {\partial z_{t}}. W$$ 

* ##### W gradient at time (t-1):
    + Repeating the same as above, we get the gradient at W as:
    
     $$\frac{\partial E_{t+1}} {\partial W} =
     \frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    \small W . 
    \frac{\partial s_{t}} {\partial z_{t}}. W. \frac{\partial s_{t}} {\partial z_{t}}. S_{t-2}$$ 
    
So the final gradient is 

$$\begin{align}\frac{\partial E_{t+1}} {\partial W} &= 
    \frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    S_{t}  \\ &+ 
    \frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    \small W . 
    \frac{\partial s_{t}} {\partial z_{t}}. S_{t-1} 
    \\ &+ 
    \frac{\partial E_{t+1}} {\partial o_{t+1}} . 
    \frac{\partial o_{t+1}} {\partial q_{t+1}} . 
    \small V . 
    \frac{\partial s_{t+1}} {\partial z_{t+1}} . 
    \small W . 
    \frac{\partial s_{t}} {\partial z_{t}}. W. \frac{\partial s_{t}} {\partial z_{t}}. S_{t-2} \end{align}$$

### Implementing in python and verifying using tensorflow

In [8]:

W = np.array([[0.4, 0.5], [0.1, 0.2]], dtype=np.float32)
print(W.shape)

(2, 2)


In [21]:
import numpy as np

# Initialize the weights
hidden_dim = 2
word_dim = 3
bptt_truncate = 3
U = np.array([[0.1, 0.2, 0.3],[0.4, 0.5, 0.6]], dtype=np.float32)
V = np.array([[0.2, 0.3],[0.5, 0.6],[0.7, 0.8]], dtype=np.float32)
W = np.array([[0.4, 0.5], [0.1, 0.2]], dtype=np.float32)
def softmax(x):
    e_x = np.exp(x)
    return e_x / e_x.sum()

def forward_propagation(x):
    T = len(x)
    s = np.zeros((T+1, hidden_dim))
    s[-1] = np.zeros(hidden_dim)
    o = np.zeros((T, word_dim))
    
    for t in np.arange(T):
        s[t] = np.tanh(U.dot(x[t]) + W.dot(s[t-1]))
        o[t] = softmax(V.dot(s[t]))
    return [o, s]

def bptt(x, y):
    T = len(y)
    # Perform forward propagation
    o, s = forward_propagation(x)
    # We accumulate the gradients in these variables
    dLdU = np.zeros(U.shape)
    dLdV = np.zeros(V.shape)
    dLdW = np.zeros(W.shape)
    delta_o = o
    delta_o[np.arange(len(y)), y] -= 1.
    # For each output backwards...
    for t in np.arange(T)[::-1]:
        dLdV += np.outer(delta_o[t], s[t].T)
        # Initial delta calculation: dL/dz
        delta_t = V.T.dot(delta_o[t]) * (1 - (s[t] ** 2))
        # Backpropagation through time (for at most self.bptt_truncate steps)
        for bptt_step in np.arange(max(0, t-bptt_truncate), t+1)[::-1]:
            # print "Backpropagation step t=%d bptt step=%d " % (t, bptt_step)
            # Add to gradients at each previous step
            dLdW += np.outer(delta_t, s[bptt_step-1])              
            dLdU[:,x[bptt_step]] += delta_t
            # Update delta for next step dL/dz at t-1
            delta_t = self.W.T.dot(delta_t) * (1 - s[bptt_step-1] ** 2)
    return [dLdU, dLdV, dLdW]

  dWxh, dWhh, dWhy = np.zeros_like(Wxh), np.zeros_like(Whh), np.zeros_like(Why)
  dbh, dby = np.zeros_like(bh), np.zeros_like(by)
  dhnext = np.zeros_like(hs[0])
  for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)

In [38]:
x = [[0, 1, 0], [1, 0, 0], [0, 0, 1]]
y = [[1, 0, 0], [0, 0, 1], [0, 1, 0]]

dU, dV, dW = bptt(x, y)
#o, s = forward_propagation([[0, 1, 2], [1, 2, 3], [4, 5, 6]])

ValueError: operands could not be broadcast together with shapes (2,3) (2,) (2,3) 

In [37]:
for t in reversed(xrange(len(inputs))):
    dy = np.copy(ps[t])
    dy[targets[t]] -= 1 # backprop into y. see http://cs231n.github.io/neural-networks-case-study/#grad if confused here
    dWhy += np.dot(dy, hs[t].T)
    dby += dy
    dh = np.dot(Why.T, dy) + dhnext # backprop into h
    dhraw = (1 - hs[t] * hs[t]) * dh # backprop through tanh nonlinearity
    dbh += dhraw
    dWxh += np.dot(dhraw, xs[t].T)
    dWhh += np.dot(dhraw, hs[t-1].T)
    dhnext = np.dot(Whh.T, dhraw)
  for dparam in [dWxh, dWhh, dWhy, dbh, dby]:
    np.clip(dparam, -5, 5, out=dparam) # clip to mitigate exploding gradients
  return loss, dWxh, dWhh, dWhy, dbh, dby, hs[len(inputs)-1]

array([0, 1, 2])

In [36]:
delta_o[np.arange(len(y)), y] -= 1.

IndexError: index 4 is out of bounds for axis 1 with size 3

In [27]:
x = [[0, 1, 2], [1, 2, 3], [4, 5, 6]]
z = np.tanh(np.dot(U, x[0]) + np.dot(W, s[-1]))
z = np.tanh(np.dot(U, x[1]) + np.dot(W, z))
print(z)
z = np.tanh(np.dot(U, x[2]) + np.dot(W, z))
print(z)

[ 0.97233045  0.99800043]
[ 0.99943743  0.99999977]


array([ 0.66403679,  0.93540908])